# Nada AI Inference Template

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NillionNetwork/nada-ai/blob/main/templates/inference_template.ipynb)

This notebook provides a generic & easily customizable end-to-end template for running AI model inference on the Nillion network.

Feel free to customize this template to fit your use case by navigating to the cells annotated by a 📝 TODO symbol!

We are really excited for developers to build with our SDK, if you have any questions please do reach out to us on:

[![Discord](https://img.shields.io/badge/Discord-nillionnetwork-%235865F2?logo=discord)](https://discord.gg/nillionnetwork)
[![GitHub Discussions](https://img.shields.io/badge/GitHub_Discussions-NillionNetwork-%23181717?logo=github)](https://github.com/orgs/NillionNetwork/discussions)

# 1. Set up environment

The boring part!

Installs all required dependencies and spins up a local devnet that will run Nada programs

In [1]:
%%capture
!pip install nada-ai~=0.3.0

In [2]:
import os
import time
import sys
import uuid

from pathlib import Path
from IPython.core.magic import register_cell_magic

In [3]:
os.makedirs("target/", exist_ok=True)

In [4]:
@register_cell_magic
def to_file(line, cell):
    "Writes the content of the cell to a file specified in the line argument."
    filepath = Path(line.strip())
    filepath.parent.mkdir(parents=True, exist_ok=True)

    with open(filepath, "w") as f:
        f.write(cell)

In [5]:
# Configure telemetry settings
enable_telemetry = True  #@param {type:"boolean"}
my_identifier = "your-telemetry-identifier"  #@param {type:"string"}

In [6]:
# Install the nilup tool and then use that to install the Nillion SDK
!curl https://nilup.nilogy.xyz/install.sh | bash

# Update Path if ran in colab
if "google.colab" in sys.modules:
    os.environ["PATH"] += ":/root/.nilup/bin"
    os.environ["PATH"] += ":/root/.nilup/sdks/latest/"

# Set telemetry if opted in
if enable_telemetry:
    identifier = f"nada-ai-inference-{str(uuid.uuid4())}-{my_identifier}"
    !echo 'yes' | nilup instrumentation enable --wallet {identifier}

# Install the lastest SDK and initialise it
!nilup init
!nilup install 0.4.0
!nilup use 0.4.0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7810  100  7810    0     0  11984      0 --:--:-- --:--:-- --:--:-- 11978

nilup has been installed into /root/.nilup/bin and added to your $PATH in /root/.bashrc.

Run 'source /root/.bashrc' or start a new terminal session to use nilup.

By providing your Ethereum wallet address, you consent to the collection of telemetry data by the Nillion Network.
That includes but is not limited to
- The version of the SDK you are using
- The OS you are using
- The Processor Architecture you are using
- The SDK binary that you are running and the subcommand
- The wallet address you provided
- The errors produced by the SDK
We collect this data to understand how the software is used, and to better assist you in case of issues.
While we will not collect any personal information, we still recommend using a new wallet address that cannot be 

In [7]:
# Spin up local Nillion devnet
!nohup nillion-devnet &

time.sleep(20)  # Wait for devnet

nohup: appending output to 'nohup.out'


# 2. Create a trained model

In this next step we will create a trained model. Either by training it ourselves or by loading a pre-trained model

In [8]:
import torch
import torch.optim as optim
import pandas as pd

from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [9]:
class MyDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        target: str="label",
    ) -> None:
        super().__init__()
        self.data = self.preprocess_data(data)

        self.features = self.data.drop(target, axis=1).values.astype(float)
        self.targets = self.data[target].values.astype(float)

    def preprocess_data(self, data: pd.DataFrame) -> pd.DataFrame:
        # TODO: 📝 define your own preprocessing logic here
        data.drop("Name", axis=1, inplace=True)
        data.rename(columns={"Survived": "label"}, inplace=True)
        data["Age"] = data["Age"].fillna(data["Age"].mean())
        data = pd.get_dummies(data, dtype="int64")
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        return feature, target

# TODO: 📝 read your own dataset!
raw_df = pd.read_csv(
    "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"
)

train_df, test_df = train_test_split(
    raw_df,
    test_size=0.20,
    random_state=42,
)

train_dataset = MyDataset(train_df)
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
)

test_dataset = MyDataset(test_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
)

In [10]:
# TODO: 📝 make your own model or load a pre-trained one

class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.ln1 = nn.Linear(7, 16)
        self.ln2 = nn.Linear(16, 1)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ln1(x)
        x = self.act(x)
        x = self.ln2(x)
        return x

my_model = MyModel()

In [11]:
training_args = {
    "num_train_epochs": 10,
    "learning_rate": 0.01,
}

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(
    my_model.parameters(),
    lr=training_args["learning_rate"],
)

for epoch in range(training_args["num_train_epochs"]):
    my_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = my_model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.6f}')

    my_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = torch.sigmoid(my_model(inputs))
            predicted = torch.round(outputs).squeeze(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.4f}%')

Epoch 1, Loss: 0.682934
Validation Accuracy: 73.5955%
Epoch 2, Loss: 0.503592
Validation Accuracy: 66.2921%
Epoch 3, Loss: 0.503470
Validation Accuracy: 74.1573%
Epoch 4, Loss: 0.494351
Validation Accuracy: 72.4719%
Epoch 5, Loss: 0.497334
Validation Accuracy: 75.2809%
Epoch 6, Loss: 0.471730
Validation Accuracy: 73.0337%
Epoch 7, Loss: 0.466857
Validation Accuracy: 75.2809%
Epoch 8, Loss: 0.460453
Validation Accuracy: 74.1573%
Epoch 9, Loss: 0.448872
Validation Accuracy: 74.7191%
Epoch 10, Loss: 0.473686
Validation Accuracy: 74.7191%


# 3. Create Nada program that runs blind inference

In [12]:
%%to_file nada-project.toml
name = "inference"
version = "0.1.0"
authors = [""]

[[programs]]
path = "src/inference.py"
prime_size = 128

In [13]:
%%to_file src/inference.py
import nada_numpy as na

from nada_ai import nn
from nada_dsl import Output
from typing import List

INPUT_DIMS=(7,)

# TODO: 📝 implement your model using nada-ai modules
class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.ln1 = nn.Linear(7, 16)
        self.ln2 = nn.Linear(16, 1)
        self.act = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        x = self.ln1(x)
        x = self.act(x)
        x = self.ln2(x)
        return x

def nada_main() -> List[Output]:
    """Main Nada program"""
    parties = na.parties(2)

    my_model = MyModel()

    my_model.load_state_from_network(
        "my_model",
        parties[0],
        na.SecretRational,
    )

    my_input = na.array(
        INPUT_DIMS,
        parties[1],
        "my_input",
        na.SecretRational,
    )

    result = my_model(my_input)

    return result.output(parties[1], "my_output")

In [14]:
!nada build

Building program: inference
Build complete!


# 4. Deploy our Nada program

In [15]:
import nada_numpy.client as na_client
from dotenv import load_dotenv
from nillion_python_helpers import create_nillion_client, create_payments_config, get_quote, get_quote_and_pay, pay_with_quote
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

In [16]:
home = os.getenv("HOME")
load_dotenv(f"{home}/.config/nillion/nillion-devnet.env")

cluster_id = os.getenv("NILLION_CLUSTER_ID")
grpc_endpoint = os.getenv("NILLION_NILCHAIN_GRPC")
chain_id = os.getenv("NILLION_NILCHAIN_CHAIN_ID")
seed = "my_seed"

userkey = UserKey.from_seed((seed))
nodekey = NodeKey.from_seed((seed))

client = create_nillion_client(userkey, nodekey)
party_id = client.party_id
user_id = client.user_id

party_names = na_client.parties(2)
program_name = "inference"
program_mir_path = f"target/{program_name}.nada.bin"

payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [17]:
quote_store_program = await get_quote(
    client, nillion.Operation.store_program(program_mir_path), cluster_id
)

receipt_store_program = await pay_with_quote(
    quote_store_program, payments_wallet, payments_client
)

action_id = await client.store_program(
    cluster_id, program_name, program_mir_path, receipt_store_program
)

program_id = f"{user_id}/{program_name}"

print("Program deployed!")
print("program_id:", program_id)

Getting quote for operation...
Submitting payment receipt 2 unil, tx hash BBBFE5266924F8E6FEEF55D07001E21D4103092E981296462559F5F3362E80BE
Program deployed!
program_id: 3rgqxWd47e171EUwe4RXP9hm45tmoXfuF8fC52S7jcFoQTnU8wPiL7hqWzyV1muak6bEg7iWhudwg4t2pM9XnXcp/inference


# 5. Provide data

In [18]:
import numpy as np
import nada_numpy as na
from nada_ai.client import TorchClient

In [19]:
permissions = nillion.Permissions.default_for_user(client.user_id)
permissions.add_compute_permissions({client.user_id: {program_id}})

In [20]:
model_client = TorchClient(my_model)
model_secrets = nillion.NadaValues(
    model_client.export_state_as_secrets("my_model", na.SecretRational)
)

receipt_store_model = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(model_secrets, ttl_days=1),
    payments_wallet,
    payments_client,
    cluster_id,
)

model_store_id = await client.store_values(cluster_id, model_secrets, permissions, receipt_store_model)

Getting quote for operation...
Quote cost is 13922 unil
Submitting payment receipt 13922 unil, tx hash C9F3EBE12D3C259FFD2B8EB35350A0A4D9C549727F2E4A149D2052E5B75B36C1


In [21]:
# TODO: 📝 provide your own input data
input_data = np.ones((7,))

my_input = na_client.array(input_data, "my_input", na.SecretRational)
input_secrets = nillion.NadaValues(my_input)

receipt_store_data = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(input_secrets, ttl_days=1),
    payments_wallet,
    payments_client,
    cluster_id,
)

data_store_id = await client.store_values(cluster_id, input_secrets, permissions, receipt_store_data)

Getting quote for operation...
Quote cost is 674 unil
Submitting payment receipt 674 unil, tx hash 32168C1B1D32D1603F151C47DD0293B49D9AD19E6CFE3F65340EA01675324AD7


# 6. Run inference

In [22]:
compute_bindings = nillion.ProgramBindings(program_id)

for party_name in party_names:
    compute_bindings.add_input_party(party_name, party_id)

compute_bindings.add_output_party(party_names[-1], party_id)

In [23]:
computation_time_secrets = nillion.NadaValues({})

receipt_compute = await get_quote_and_pay(
    client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

_ = await client.compute(
    cluster_id,
    compute_bindings,
    [model_store_id, data_store_id],
    computation_time_secrets,
    receipt_compute,
)

while True:
    compute_event = await client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅ Compute complete for compute_id {compute_event.uuid}")
        result = compute_event.result.value
        break

result = {
    key: na_client.float_from_rational(value)
    for key, value in result.items()
}
print("Result is", result)

Getting quote for operation...
Quote cost is 388 unil
Submitting payment receipt 388 unil, tx hash 92FEB37AC48304A94A49C47431324903161280795FDDCFF64572CABB3B268A15
✅ Compute complete for compute_id 3b8b0293-a7a0-49ec-8821-f32913ddfd80
Result is {'my_output_0': -0.1307373046875}
